# Initialization
Select the Julia kernel

#### Initialize

In [4]:
using Statistics
using Distributions
using LinearAlgebra
using QuantEcon # http://quantecon.github.io/QuantEcon.jl/latest/api/QuantEcon.html

# Usefull functions

## n-step probabilities

In [34]:
P = Matrix([0.75 0.2 0.05; 0.3 0.6 0.1; 0.15 0.05 0.8])
# 5 steps:
P^5

3×3 Matrix{Float64}:
 0.515874  0.293803  0.190323
 0.485464  0.293036  0.221501
 0.39193   0.200269  0.4078

## t-test

In [ ]:
c = [1.85,1.92,1.95,1.7,2.1,2.5,2.2,2,2.08]

function t_test(x; conf_level=0.95)
    alpha = (1 - conf_level)
    tstar = quantile(TDist(length(x)-1), 1 - alpha/2)
    SE = std(x)/sqrt(length(x))

    lo = Statistics.mean(x) + (-1 * tstar * SE)
    hi = Statistics.mean(x) + 1 * tstar * SE
    "($lo, $hi)"
end


t_test(c)

# Classification of states

In [10]:
P = [0 1 0; 0 0 1; 1 0 0]
mc = MarkovChain(P)

Discrete Markov Chain
stochastic matrix of type Matrix{Int64}:
[0 1 0; 0 0 1; 1 0 0]

### Class

A collection of states that communicates together.

### Transient state

Upon entering this state, the process might never return to this state. Thus a state i is transient when
there exists a state j that is accessible from i, but not
vice versa, that is, i is not accessible from j.

### Recurrent state

Upon entering this state, the process definetely will return to this state. A state is recurrent when it is not
transient. A recurrent state will be visited an infinite number of times.

### Absorbing state

Upon entering this state, the process will never leave this state. A state is an absorbing state if and only if pii = 1. Absorbing states are a specific type of recurrent states.

### Irreducibility

In [4]:
# QuantEcon
print(is_irreducible(mc))

true

A Markov chain is irreducible when it containts only one class.

### Comunicating classes

In [8]:
# QuantEcon
communication_classes(mc)

2-element Vector{Vector{Int64}}:
 [1]
 [2, 3]

wo states i and j are said to communicate when i is accessible from j and j is accessible from i.

* Any state communicates with itself
*  If state i communicates with j, then state j communicates with i.
* If state i communicates with j, and state j communicates with k, then state i communicates with k

### Recurrent Classes

In [14]:
# QuantEcon
recurrent_classes(mc)

1-element Vector{Vector{Int64}}:
 [1, 2, 3]

Upon entering this state, the process definetely will return to this state. A state is recurrent when it is not
transient. A recurrent state will be visited an infinite number of times.

### Aperiodicity

In [11]:
# QuanEcon
period(mc) # returns periods
is_aperiodic(mc) # True/False

3

Has a period of 1. That is any process that has two consecutive integers s, s+1 such that it can be in this state at times s and s + 1.

### Ergodic state

Recurrent states that are aperiodic in a finite state Markov chain.

# Steady State

Steady state probability:
The steady state is the long term probabilities as in Pi_j=limit(p_ij^n, n->infinity) where Pi_j > 0

Requirements:
Ergodic and irreducible (A Markov chain is ergodic if and only if it has at most one recurrent class and is aperiodic.)

Calculate:
Pi_j=sum(Pi_i*p_ij,i=0..M)

In [12]:
# Choose your matrix
P = Matrix([0.75 0.2 0.05; 0.3 0.6 0.1; 0.15 0.05 0.8])

# Mine
function steady(P)
    mc = MarkovChain(P)
    if is_irreducible(mc) == false # also check for ergotic
        throw(ErrorException("Is not Irreducible"))
    elseif length(recurrent_classes(mc)) > 1 || is_aperiodic(mc) == false
        throw(ErrorException("Is not Ergodic"))
    end
    
    # Computing the left side of the equation
    A=P'-Matrix(1.0I, length(P[1,:]), length(P[:,1]))

    ones = repeat([1.0],length(P[1,:]))'
    A = vcat(A,ones)
    # The right side of the equation
    b = vcat(repeat([0.0],length(A[1,:])),1.0)

    #Probability to be in a certain state
    Pi = A\b
end

# QuanEcon
mc = MarkovChain(P);
stationary_distributions(mc)

1-element Vector{Vector{Float64}}:
 [0.47619047619047616, 0.2698412698412698, 0.253968253968254]

# Absorbing States

Absorbing Probabilities are determined as: B = (I - Q)^-1 * R or P= [ [Q,R] [0,I] ]
where Q is a t-by-t matrix, R is a nonzero t-by-r matrix, 0 is an r-by-t zero matrix, and I is the r-by-r identity matrix. Thus, Q describes the probability of transitioning from some transient state to another while R describes the probability of transitioning from some transient state to some absorbing state. Source: https://en.wikipedia.org/wiki/Absorbing_Markov_chain

In [8]:

P = Matrix([1 0 0 0; 0.7 0 0.3 0; 0 0.7 0 0.3; 0 0 0 1])

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.7  0.0  0.3  0.0
 0.0  0.7  0.0  0.3
 0.0  0.0  0.0  1.0

### Initialize Absorbing States

The (i, j) entry of matrix N is the expected number of times the chain is in state j, given that the chain started in state i. N is obtained with Q

In [25]:
####
# Creating the Fundamental matrix N = (I-Q)^-1
# This can be used for practically this entire area of markov

function num_trans(P)
    t = []
    r = []
    a = 0
    party = 1
    # Loop through matrix
    for k in 1:length(P[:,1])
        a += 1

        for c in 1:length(P[1,:])
            if P[k,c] == 1
                # If there is a prob of 1 to return to same state it absorbs and is not included.
                append!(r,a)
                party = 0
                break
            end
        end
        # Not decent code at all, but it works..
        if party == 1
            append!(t,a)
        else
            party = 1
        end
    end
    return t, r
end

function create_RQ(P)
    t,r = num_trans(P)

    # Creating empty vector 
    Q = Vector{Float64}(undef,length(t))'
    # Choosing all the transient states into a pseydo transient matrix by size t-t
    for k in t
        qRow = []
        for c in t
            append!(qRow,P[k,c])
        end
        Q = vcat(Q, qRow')
    end

    R = Vector{Float64}(undef,length(r))'
    
    if isempty(r)
        # Doing the same but for absorbant states into t-r, a few changes in the way the matrices combine
        return Q[2:length(t)+1,:], R
    else
        for k in t
            rRow = []
            for c in r
                append!(rRow,P[k,c])
            end
            R = vcat(R, rRow')
        end
        R = R[2:length(t)+1,:]
    end

    return Q[2:length(t)+1,:], R
end

# # Constructing the full baby, not at all necesary haha
# begin
# Q,R = create_RQ(P)
# hcat(zeros(2,3),Matrix(I,2,2))
# hcat(Q,R)
# vcat(hcat(Q,R),hcat(zeros(2,3),Matrix(I,2,2)))
# end




create_RQ (generic function with 1 method)

### Absorbing Probabilities

Another property is the probability of being absorbed in the absorbing state j when starting from transient state i, which is the (i,j)-entry of the matrix B = N R

They use the sum methode again with condition:

f_kk = 1
f_ik = 0 if state i is recurrent and i not equal k

In [26]:
Q,R = create_RQ(P)
N = inv(Matrix(1.0I,length(Q[1,:]),length(Q[:,1])) .- Q)
B = N*R

Any[0.7 0.0; 0.0 0.3][1.2658227848101267 0.37974683544303794; 0.8860759493670884 1.2658227848101264]

2×2 Matrix{Any}:
 0.886076  0.113924
 0.620253  0.379747

### Expected number of steps before absorption

In [6]:
# Expected number of steps before being absorbed, done from the wiki article
function ENS(P)
    Q,R = create_RQ(P)
    N = inv(1*Matrix(I,length(Q[1,:]),length(Q[:,1])) .- Q)
    return N*ones(3)
end
ENS(P)


3-element Vector{Float64}:
 3.2
 3.2
 2.4000000000000004

### Transient Probabilities

The probability of visiting transient state j when starting at a transient state i is the (i,j)-entry of the matrix H = (N-I)(Ndg)^-1

In [27]:
function TP(P)
    Q,R = create_RQ(P)
    N = inv(Matrix(1.0I,length(Q[1,:]),length(Q[:,1])) .- Q)

    H = (N-Matrix(1.0I,length(N[1,:]),length(N[:,1])))*inv(Diagonal(N))
end

TP(P)


2×2 Matrix{Float64}:
 0.21  0.3
 0.7   0.21

# First Passage Time

#### Mean Number of Steps, Hitting Time or First Passage Time

Ok. This does not follow the method in the book or lectures. It takes a linear algebra approach, actualy the same is acomplished but with matrices and not individual calculations.
It follows this guide and proof, if you want to go deeper: https://stats.libretexts.org/Bookshelves/Probability_Theory/Book%3A_Introductory_Probability_(Grinstead_and_Snell)/11%3A_Markov_Chains/11.05%3A_Mean_First_Passage_Time_for_Ergodic_Chains
Alright, it creates a fundamental matrix for the ergodic chain, much like the absorbing earlier, then it uses the steady probabilities and the difference between two points on the fundamental.
What they do is just set it as three equations and solve them:

mu_30 = 1 + p_10*mu_10 + p_20*mu_20 + p_30*mu_30

mu_20 = 1 + p_10*mu_10 + p_20*mu_20 + p_30*mu_30

mu_10 = 1 + p_10*mu_10 + p_20*mu_20 + p_30*mu_30

Then solve.
The below methode will generelize better and give a matrix for all possible steps. 

In [13]:
# Initialize steady function

function MeanNumSteps(P)
    l = length(P[1,:])

    # Creates a matrix with the steady states, terrible code but it works
    W = repeat(vcat(steady(P)',steady(P)'),Int(round(l/2)))
    if isodd(l)
        W = W[1:l,:]
    end

    # Fundamental Matrix
    Z = inv(Matrix(1.0I,l,l) - P + W)

    # vector of steady states to use for the transition
    w = steady(P)

    # Empty vector, have not found a way to concenate without a starting row of same size
    M = Vector{Float64}(undef,l)'
    # The magic happens here!
    # It creates 
    for k in 1:l
        mRow = []
        for c in 1:l
            a = (Z[k,k] - Z[c,k])/w[k]
            append!(mRow,a)
        end
        M = vcat(M, mRow')
    end
    return M[2:l+1,:]' + Matrix(1.0I,l,l)
end

MeanNumSteps (generic function with 1 method)

In [14]:
P = Matrix([0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25; 0.5 0.25 0.25 0; 0.25 0.25 0.25 0.25])
MeanNumSteps(P)

4×4 Matrix{Float64}:
 1.0  4.0  4.0  5.33333
 3.2  1.0  4.0  5.33333
 2.4  4.0  1.0  6.66667
 3.2  4.0  4.0  1.0

#### Expected Recurence Time

The steps it takes to return to the original state.
Not a usefull function in itself but helpfull to remember how to do it.

In [30]:

function ERT(P,i)
    # takes transition matrix and state
    Pi = steady(P)
    1/Pi[i]
end

ERT (generic function with 1 method)

Ergodicity:
ergodicity expresses the idea that a point of a moving system, either a dynamical system or a stochastic process, will eventually visit all parts of the space that the system moves in. Pi= Pi*p, Pi_i >0



## n step probability

Probability of moving from i to j in n steps.

In [2]:


function nStepProp(P,i,j,n)
    # Takes transition matrix: P
    # i: start state
    # j: end state
    # n: steps
    n=n-1

    # Creating the vectors for the used probabilities
    # Since k cannot equal j 
    p = []
    f = []
    for k in 1:length(P[1,:])
        if k != j
            append!(p,P[i,k])
            append!(f,P[k,j])
        end
    end

    for _ in 1:n
        #Running the iterative function
        f = p.*f
    end
    #Final Probability
    prop = sum(f)
    return prop
end

nStepProp (generic function with 1 method)

In [3]:
P = [0 1 0 0; 0.6 0 0.4 0; 0 0.6 0 0.4; 0 0 1 0]
nStepProp(P,1,3,3)

0.4